In [9]:
import re
import os
import subprocess
source = "IITG"
# destination = ["google.txt","apple.txt","bitcoin.txt","goldmansachs.txt","ntt.txt","spotify.txt","vodacom.txt","wikipedia.txt","oracle.txt"]
# destination = ["google.txt","apple.txt","bitcoin.txt","ntt.txt","spotify.txt"]
destination = ["google.txt","alibaba.txt","bitcoin.txt","ntt.txt","spotify.txt","amazon.txt"]
# destination = ["nvidia.txt"]
def getdata(source,destination):
    file = open(source+"/"+destination,"r")
    data = {
        "AS NUMBER" : [],
        "IP ADDRESS": [],
        "IP RANGE" : [],
        "LOCATION" : [],
        "ORGANISATION": []
    }
    for line in file.readlines():

    
        s = re.findall('\[AS[0-9]+',line)
        
        if (len(s)>0 and s[0][1] != "*"):
            
            ips = re.findall('[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+',line)

            statement = "./IPGeoLocation/ipgeolocation.py -t {} ".format(ips[0])
            proc = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc.communicate()
            out = out.decode()
            city = re.findall('City:[^\n]*\\n',out)
            city = re.findall('[A-Z][a-zA-Z ]*',city[0])
            country= re.findall('Country:[^\n]*\\n',out)
            country = re.findall('[A-Z][a-zA-Z ]*',country[0])
            org= re.findall('Organization:[^\n]*\\n',out)
            org = re.findall('[A-Z][a-zA-Z ]*',org[0])
            if (len(org) >=2):
                org = org[1]
            else:
                org = ""
            if (len(city) >=2):
                city = city[1]
            else:
                city = ""
            if (len(country) >=2):
                country = country[1]
            else:
                country = ""
            
            loc = city+'/'+country
            statement = "whois -a {}".format(ips[0])
            proc2 = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc2.communicate()
            out = out.decode()
            iprange = re.findall('inetnum:[ \t]*[-0-9\. ]*|NetRange:[ \t]*[-0-9\. ]*',out)
            if (len(iprange) > 0):
                iprange = iprange[0].split(":")[1].lstrip()
            else:
                iprange = ""
                
            
            data["IP RANGE"].append(iprange)
            data["AS NUMBER"].append(s[0][1:])
            data["IP ADDRESS"].append(ips[0])
            data["ORGANISATION"].append(org)
            data["LOCATION"].append(loc)
            
    file.close()
    return data


In [10]:
# getdata(source,destination[0])
destination_dict = {}
for d in destination:
    print(d)
    destination_dict[d] = getdata(source,d)
destination_dict

google.txt
alibaba.txt
bitcoin.txt
ntt.txt
spotify.txt
amazon.txt


{'google.txt': {'AS NUMBER': ['AS0',
   'AS0',
   'AS0',
   'AS0',
   'AS9498',
   'AS55536',
   'AS15169',
   'AS15169',
   'AS15169',
   'AS15169'],
  'IP ADDRESS': ['172.20.10.1',
   '192.168.27.237',
   '192.168.27.5',
   '192.168.31.217',
   '182.78.193.13',
   '116.119.73.82',
   '72.14.213.254',
   '142.251.225.9',
   '142.250.214.109',
   '142.250.183.14'],
  'IP RANGE': ['172.16.0.0 - 172.31.255.255',
   '192.168.0.0 - 192.168.255.255',
   '192.168.0.0 - 192.168.255.255',
   '192.168.0.0 - 192.168.255.255',
   '182.64.0.0 - 182.79.255.255',
   '116.119.0.0 - 116.119.127.0',
   '72.14.192.0 - 72.14.255.255',
   '142.250.0.0 - 142.251.255.255',
   '142.250.0.0 - 142.251.255.255',
   '142.250.0.0 - 142.251.255.255'],
  'LOCATION': ['/',
   '/',
   '/',
   '/',
   'Kolkata/India',
   'Gurugram/India',
   'Mountain View/United States',
   'Mountain View/United States',
   'Lakeland/United States',
   'Hackettstown/United States'],
  'ORGANISATION': ['',
   '',
   '',
   '',
   'Bha

In [11]:
destination_dict

{'google.txt': {'AS NUMBER': ['AS0',
   'AS0',
   'AS0',
   'AS0',
   'AS9498',
   'AS55536',
   'AS15169',
   'AS15169',
   'AS15169',
   'AS15169'],
  'IP ADDRESS': ['172.20.10.1',
   '192.168.27.237',
   '192.168.27.5',
   '192.168.31.217',
   '182.78.193.13',
   '116.119.73.82',
   '72.14.213.254',
   '142.251.225.9',
   '142.250.214.109',
   '142.250.183.14'],
  'IP RANGE': ['172.16.0.0 - 172.31.255.255',
   '192.168.0.0 - 192.168.255.255',
   '192.168.0.0 - 192.168.255.255',
   '192.168.0.0 - 192.168.255.255',
   '182.64.0.0 - 182.79.255.255',
   '116.119.0.0 - 116.119.127.0',
   '72.14.192.0 - 72.14.255.255',
   '142.250.0.0 - 142.251.255.255',
   '142.250.0.0 - 142.251.255.255',
   '142.250.0.0 - 142.251.255.255'],
  'LOCATION': ['/',
   '/',
   '/',
   '/',
   'Kolkata/India',
   'Gurugram/India',
   'Mountain View/United States',
   'Mountain View/United States',
   'Lakeland/United States',
   'Hackettstown/United States'],
  'ORGANISATION': ['',
   '',
   '',
   '',
   'Bha

In [13]:
import pandas as pd

for d in destination:
    df = pd.DataFrame(destination_dict[d])
    df.to_excel(source +"/"+"structured" + "/" + d[:-3]+"xlsx")